In [ ]:
!pip install paddlepaddle-gpu -q
!pip install paddleocr -q

In [ ]:
from paddleocr import PaddleOCR
import cv2
model_url = "https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_det_train.tar"

# Create an OCR instance using the specified model
ocr_model = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True,rec_model_url=model_url)

In [ ]:
!wget -O "ML.zip" "https://www.dropbox.com/scl/fi/j3n22ahtoa3sf96phyzgm/66e31d6ee96cd_student_resource_3.zip?rlkey=pthkphkleyzzvvsgrzfbexpf7&st=pk1vscuk&dl=0" -q
!unzip -o -q "ML.zip"

In [ ]:
import pandas as pd
data=pd.read_csv('./student_resource 3/dataset/test.csv')

In [ ]:
dp_area={}

In [ ]:
# with open('dp_area.pkl', 'rb') as f:
#     dp_area = pickle.load(f.read()) #cached 1000images
def perf_ocr_area(img_path):
    if img_path in dp_area:
        return dp_area[img_path]
    image=cv2.imread(img_path)
    result = ocr_model.ocr(image, cls=True)
    res = []  # List to store tuples of (area, text, original_index)
    
    try:
        for idx in range(len(result)):
            res_lines = result[idx]
            for line in res_lines:
                # Extract the bounding box coordinates from `line`
                box = line[0]
                width = abs(box[1][0] - box[0][0])  # x2 - x1
                height = abs(box[2][1] - box[1][1])  # y3 - y2
                area_rect = width * height  # Calculate the area

                # Extract the recognized text from `line`
                text = line[-1][0]

                # Append the tuple (area, text, original_index) to the `res` list
                res.append((area_rect, text, len(res)))  # `len(res)` gives the current index

        # Sort the list in descending order based on the area
        sorted_res = sorted(res, key=lambda x: x[0], reverse=True)

        # Create sorted_one and original_index arrays
        sorted_one = [(area, text, orig_idx) for area, text, orig_idx in sorted_res]
        original_order = [(area, text, orig_idx) for area, text, orig_idx in res]

        # Store the result in dp_area
        dp_area[img_path] = (sorted_one, original_order)
    except:
        dp_area[img_path]=([],[])
        return [],[]
    return sorted_one, original_order


In [ ]:
import os
def download_image(url, folder):
    img_name = url.split('/')[-1]
    img_path = os.path.join(folder, img_name)
    if not os.path.exists(img_path):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                with open(img_path, 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Failed to download {url}: HTTP {response.status_code}")
        except Exception as e:
            print(f"Error downloading {url}: {e}")

def download_images_concurrently(image_urls, folder):
    os.makedirs(folder, exist_ok=True)
    with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
        futures = [executor.submit(download_image, url, folder) for url in image_urls]
        for future in concurrent.futures.as_completed(futures):
            future.result()  # To catch exceptions

            
for i in range(0,len(data),1000):
    os.makedirs('images',exist_ok=True)
    # # Run the asynchronous pipeline
    temp_data = data.iloc[i:i+1000]
    image_urls = temp_data['image_link']
    folder = 'images'
    try:
        download_images_concurrently(image_urls, folder)
    except:
        zz=1
#     try:    
    for j in image_urls:
        try:
            img_name=j.split('/')[-1]
            img_path=f"./images/{img_name}"
            print(img_path)
            if not os.path.exists(img_path):
                !wget -O {img_path} {j} -q
            perf_ocr_area(img_path)
            print(1)
        except:
            continue
#     except:
#         continue
    !rm -rf "./images"

In [ ]:
len(dp_area)

In [ ]:
import pickle

with open('dp_area.pkl', 'wb') as file:
    pickle.dump(dp_area, file)